In [2]:
!pwd


/home/foivos/Sage/Pollack_tables/plus_minus


In [7]:
import pandas as pd
import csv

p = 11
file_path = f"../{p}_lam_not_1_df.csv"

# Columns you want to keep, parameterized by p
columns_to_keep = [
    "Conductor",
    "Rank",
    f"mu_{p}",
    f"lam_{p}",
    f"lam_{p}_MW"
]

df = pd.read_csv(file_path, usecols=columns_to_keep)

lam_series = df.groupby("Conductor")[f"lam_{p}"].agg(list)

# Indices where the first (i.e., plus) value is > 1
plus_cond = lam_series[lam_series.apply(lambda vals: vals[0] > 1)].index

# Indices where the second (i.e., minus) value is > 1
minus_cond = lam_series[lam_series.apply(lambda vals: len(vals) > 1 and vals[1] > 1)].index

# Subset of the DataFrame containing *both rows* for those conductors with lam_5^+ > 1
df_plus_bigger_one = df[df["Conductor"].isin(plus_cond)]
plus_name = f"lam_{p}_plus_big.csv"
df_plus_bigger_one.to_csv(plus_name, index=False)

# Subset of the DataFrame containing *both rows* for those conductors with lam_5^- > 1
df_minus_bigger_one = df[df["Conductor"].isin(minus_cond)]
minus_name = f"lam_{p}_minus_big.csv"
df_minus_bigger_one.to_csv(minus_name, index=False)

# Subset of the DataFrame containing *both rows* for those conductors with BOTH lam_5^+ > 1 and lam_5^- > 1
plus_and_minus_cond = lam_series[lam_series.apply(lambda vals: vals[0] > 1 and len(vals) > 1 and vals[1] > 1)].index
df_plus_and_minus_bigger_one = df[df["Conductor"].isin(plus_and_minus_cond)]
plus_and_minus_name = f"lam_{p}_plus_and_minus_big.csv"
df_plus_and_minus_bigger_one.to_csv(plus_and_minus_name, index=False)


In [5]:
import pandas as pd
import csv

p = 11
file_path = f"../{p}_lam_1_df.csv"

# Columns you want to keep, parameterized by p
columns_to_keep = [
    "Conductor",
    "Rank",
    f"mu_{p}",
    f"lam_{p}",
    f"lam_{p}_MW"
]

df = pd.read_csv(file_path, usecols=columns_to_keep)
name=f"lam_{p}_equals_1.csv"
df.to_csv(name,index=False)

In [7]:

primes = [5, 7, 11]

results = []

for p in primes:
    # Construct file names
    plus_file = f"lam_{p}_plus_big.csv"
    minus_file = f"lam_{p}_minus_big.csv"
    plus_and_minus_file = f"lam_{p}_plus_and_minus_big.csv"
    equals_1_file=f"lam_{p}_equals_1.csv"
    
    # Read each file into a DataFrame
    plus_df = pd.read_csv(plus_file)
    minus_df = pd.read_csv(minus_file)
    plus_and_minus_df = pd.read_csv(plus_and_minus_file)
    equals_1_df=pd.read_csv(equals_1_file)
    
    # Count distinct conductors
    plus_count = plus_df["Conductor"].nunique()
    minus_count = minus_df["Conductor"].nunique()
    plus_and_minus_count = plus_and_minus_df["Conductor"].nunique()
    equals_1_count=equals_1_df["Conductor"].nunique()
    
    # Save the counts in a list
    results.append({
        "p": p,
        "lam+- =1": equals_1_count,
        "lam+>1": plus_count,
        "lam->1": minus_count,
        "lam+ and lam->1": plus_and_minus_count
    })

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

results_df.head()


,p,lam+- =1,lam+>1,lam->1,lam+ and lam->1
0,5,219,72,68,8
1,7,203,33,38,1
2,11,306,31,35,1


# Computations 

In [17]:
import pandas as pd 

p=5
def ell_list(conductor_list):
    return [EllipticCurve(a) for a in conductor_list]


plus_path = f"lam_{p}_plus_big.csv"
minus_path=f"lam_{p}_minus_big.csv"

# Read only the 'Conductor' column, drop NaNs, extract unique values
df_plus = pd.read_csv(plus_path, usecols=["Conductor"]).dropna()
my_list_plus = df_plus["Conductor"].unique().tolist()

df_minus = pd.read_csv(minus_path, usecols=["Conductor"]).dropna()
my_list_minus = df_minus["Conductor"].unique().tolist()

# Create the list of EllipticCurve objects
ELL_plus = ell_list(my_list_plus)
ELL_minus = ell_list(my_list_minus)

In [22]:
def phi(E,p):
    Lp=E.padic_lseries(p)
    phi = Lp.frobenius(prec=5, algorithm="mw")
    return(phi)
    
def manual_reg(E,p):
    R1=old_valued_regulator(E,p)
    A=R1[0]
    B=R1[1]
    Phi=phi(E,p)
    a=Phi[0,0]
    c=Phi[1,0]
    b=Phi[0,1]
    d=Phi[1,1]
    return([A-a*B/c,B/c])
    
def Lp_series(E,p,prec=6):
    L=E.padic_lseries(p)
    lps = L.series(n=5, prec=6)
    R = lps.base_ring().base_ring()  # Qp
    QpT, T = PowerSeriesRing(R, 'T', 6).objgen()
    Gli = []
    Hli = []
    for n in range(lps.prec()):
        v = L._poly(lps[n])
        Gli.append(v[0])
        Hli.append(v[1])
    G = QpT(Gli, prec=6)
    H = QpT(Hli, prec=6)
    return([G,H])

In [100]:
results_plus = {}
for E in ELL_plus:
    L = E.padic_lseries(p)
    L1 = L.Dp_valued_series(n=6, prec=6)
    Phi = phi(E, p)
    L_1_original = (1 - Phi)**2 * L1
    new_reg = E.padic_regulator(p) % p^6
    Tam = E.tamagawa_product()
    G, H = Lp_series(L, p)
    results_plus[E.label()] = {
        "Lp_old": L_1_original,
        "Lp_new": L1,
        "Reg_p": new_reg,
        "Tam(E)": Tam,
        "G": G
    }

df_plus_bigger_one = df_plus_bigger_one.copy()
df_plus_bigger_one.loc[:, "Lp_old"] = df_plus_bigger_one["Conductor"].map(lambda c: results_plus[c]["Lp_old"])
df_plus_bigger_one.loc[:, "Lp_new"] = df_plus_bigger_one["Conductor"].map(lambda c: results_plus[c]["Lp_new"])
df_plus_bigger_one.loc[:, "Reg_p"]  = df_plus_bigger_one["Conductor"].map(lambda c: results_plus[c]["Reg_p"])
df_plus_bigger_one.loc[:, "Tam(E)"] = df_plus_bigger_one["Conductor"].map(lambda c: results_plus[c]["Tam(E)"])
df_plus_bigger_one.loc[:, "G"]      = df_plus_bigger_one["Conductor"].map(lambda c: results_plus[c]["G"])
df_plus_bigger_one.to_csv(plus_name, index=False)

results_minus = {}
for E in ELL_minus:
    L = E.padic_lseries(p)
    L1 = L.Dp_valued_series(n=6, prec=6)
    Phi = phi(E, p)
    L_1_original = (1 - Phi)**2 * L1
    new_reg = E.padic_regulator(p) % p^6
    Tam = E.tamagawa_product()
    G, H = Lp_series(L, p)
    results_minus[E.label()] = {
        "Lp_old": L_1_original,
        "Lp_new": L1,
        "Reg_p": new_reg,
        "Tam(E)": Tam,
        "H": H
    }

df_minus_bigger_one = df_minus_bigger_one.copy()
df_minus_bigger_one.loc[:, "Lp_old"] = df_minus_bigger_one["Conductor"].map(lambda c: results_minus[c]["Lp_old"])
df_minus_bigger_one.loc[:, "Lp_new"] = df_minus_bigger_one["Conductor"].map(lambda c: results_minus[c]["Lp_new"])
df_minus_bigger_one.loc[:, "Reg_p"]  = df_minus_bigger_one["Conductor"].map(lambda c: results_minus[c]["Reg_p"])
df_minus_bigger_one.loc[:, "Tam(E)"] = df_minus_bigger_one["Conductor"].map(lambda c: results_minus[c]["Tam(E)"])
df_minus_bigger_one.loc[:, "H"]      = df_minus_bigger_one["Conductor"].map(lambda c: results_minus[c]["H"])
df_minus_bigger_one.to_csv(minus_name, index=False)



# Testing

In [19]:
E=EllipticCurve('272a1')
L = E.padic_lseries(p)
L1 = L.Dp_valued_series(n=6, prec=6)
Phi = phi(E, p)
L1_original = (1 - Phi)**2 * L1
new_reg = E.padic_regulator(p) % p^6
Tam = E.tamagawa_product()

In [42]:
L1

((1 + 2*5 + O(5^2))*T + (3*5 + O(5^2))*T^2 + (4 + 4*5 + O(5^2))*T^3 + (1 + 4*5 + O(5^2))*T^4 + O(5)*T^5 + O(T^6), (3*5 + 3*5^2 + O(5^3))*T + (4*5^2 + O(5^3))*T^2 + (5 + 4*5^2 + O(5^3))*T^3 + (4*5 + O(5^3))*T^4 + (2*5 + O(5^2))*T^5 + O(T^6))

In [45]:
L1_original

((4*5^-1 + 3 + O(5))*T + (2 + O(5))*T^2 + (5^-1 + O(5))*T^3 + (4*5^-1 + O(5))*T^4 + O(5^0)*T^5 + O(T^6), (1 + O(5^2))*T + (3*5 + O(5^2))*T^2 + (2*5 + O(5^2))*T^3 + (4*5 + O(5^2))*T^4 + (3 + O(5))*T^5 + O(T^6))

In [78]:
M=matrix([[1,2],[3,4]])
M

[1 2]
[3 4]

In [20]:
lps = L.series(n=5, prec=6)

# now split up the series in two lps = G + H * alpha
R = lps.base_ring().base_ring()  # Qp
QpT, T = PowerSeriesRing(R, 'T', 6).objgen()
Gli = []
Hli = []
for n in range(lps.prec()):
    v = L._poly(lps[n])
    Gli.append(v[0])
    Hli.append(v[1])
G = QpT(Gli, prec=6)
H = QpT(Hli, prec=6)

In [21]:
G

(4*5^-1 + 4 + O(5))*T + (2 + O(5))*T^2 + (5^-1 + 1 + O(5))*T^3 + (4*5^-1 + 4 + O(5))*T^4 + O(T^6)

In [63]:
Lp_series(L,p)

[(4*5^-1 + 4 + O(5))*T + (2 + O(5))*T^2 + (5^-1 + 1 + O(5))*T^3 + (4*5^-1 + 4 + O(5))*T^4 + O(T^6),
 (1 + O(5))*T + O(5)*T^2 + (4*5^-1 + 1 + O(5))*T^3 + (5^-1 + O(5))*T^4 + O(T^6)]

In [65]:
G,H=Lp_series(L,p)

In [66]:
G

(4*5^-1 + 4 + O(5))*T + (2 + O(5))*T^2 + (5^-1 + 1 + O(5))*T^3 + (4*5^-1 + 4 + O(5))*T^4 + O(T^6)

In [5]:
S.<p,G,H,a,b,c,d,A,B> = PolynomialRing(QQ)
M=matrix([[0, -1 / p], [1, 0]])
W=vector([G,-p*H])
T=(1-M)^2
K=T.inverse()

In [102]:
T=(1-M)^2
W=vector([G+a*H/c, -H/c])

In [103]:
T*W

((p*H*a + p*G*c - H*a - G*c - 2*H)/(p*c), (-2*p*H*a - 2*p*G*c - p*H + H)/(p*c))

In [2]:
M1=matrix([[a,b],[c,d]])
R=vector([A,-B])

In [108]:
(1-M1)^2 * R

(a^2*A + b*c*A - a*b*B - b*d*B - 2*a*A + 2*b*B + A, a*c*A + c*d*A - b*c*B - d^2*B - 2*c*A + 2*d*B - B)

In [109]:
M2=matrix([[0, -1 / p], [1, 0]])
R2=vector([A+a*B/c, B/c])

In [110]:
(1-M2)^2 * R2

((p*c*A + p*a*B - c*A - a*B + 2*B)/(p*c), (-2*p*c*A - 2*p*a*B + p*B - B)/(p*c))

In [4]:
(1-M1)^(-2)

[(b*c + d^2 - 2*d + 1)/(b^2*c^2 - 2*a*b*c*d + a^2*d^2 + 2*a*b*c - 2*a^2*d + 2*b*c*d - 2*a*d^2 + a^2 - 2*b*c + 4*a*d + d^2 - 2*a - 2*d + 1)    (a*b + b*d - 2*b)/(-b^2*c^2 + 2*a*b*c*d - a^2*d^2 - 2*a*b*c + 2*a^2*d - 2*b*c*d + 2*a*d^2 - a^2 + 2*b*c - 4*a*d - d^2 + 2*a + 2*d - 1)]
[   (a*c + c*d - 2*c)/(-b^2*c^2 + 2*a*b*c*d - a^2*d^2 - 2*a*b*c + 2*a^2*d - 2*b*c*d + 2*a*d^2 - a^2 + 2*b*c - 4*a*d - d^2 + 2*a + 2*d - 1) (a^2 + b*c - 2*a + 1)/(b^2*c^2 - 2*a*b*c*d + a^2*d^2 + 2*a*b*c - 2*a^2*d + 2*b*c*d - 2*a*d^2 + a^2 - 2*b*c + 4*a*d + d^2 - 2*a - 2*d + 1)]

In [7]:
(1-M)^(-2)

[(p^2 - p)/(p^2 + 2*p + 1)    (-2*p)/(p^2 + 2*p + 1)]
[    2*p^2/(p^2 + 2*p + 1) (p^2 - p)/(p^2 + 2*p + 1)]

In [9]:
S.<A,B>=QQ[]
E=EllipticCurve([A,B])
p=5
R=Zmod(p^2)

In [14]:
f=E.division_polynomial(p+2,two_torsion_multiplicity=1)


In [16]:
T.<t>=Zmod(p^2)[]

In [18]:
f

7*x^24 + 308*A*x^22 + 3944*B*x^21 + (-2954*A^2)*x^20 + (-112*A*B)*x^19 + (-19852*A^3 - 42896*B^2)*x^18 + (-92568*A^2*B)*x^17 + (-35231*A^4 - 571872*A*B^2)*x^16 + (-31808*A^3*B - 829696*B^3)*x^15 + (-82264*A^5 - 615360*A^2*B^2)*x^14 + (-161840*A^4*B - 2132480*A*B^3)*x^13 + (-111916*A^6 - 297472*A^3*B^2 - 928256*B^4)*x^12 + (-608160*A^5*B - 2603776*A^2*B^3)*x^11 + (-42168*A^7 - 1192800*A^4*B^2 - 3293696*A*B^4)*x^10 + (-425712*A^6*B - 3727360*A^3*B^3 - 1555456*B^5)*x^9 + (15673*A^8 - 831936*A^5*B^2 - 7069440*A^2*B^4)*x^8 + (-53824*A^7*B - 1314560*A^4*B^3 - 7127040*A*B^5)*x^7 + (14756*A^9 - 190400*A^6*B^2 - 2293760*A^3*B^4 - 2809856*B^6)*x^6 + (57288*A^8*B - 168448*A^5*B^3 - 3698688*A^2*B^5)*x^5 + (1302*A^10 + 134400*A^7*B^2 + 394240*A^4*B^4 - 3039232*A*B^6)*x^4 + (1680*A^9*B + 152320*A^6*B^3 + 831488*A^3*B^5 - 802816*B^7)*x^3 + (196*A^11 + 3696*A^8*B^2 + 96768*A^5*B^4 + 544768*A^2*B^6)*x^2 + (392*A^10*B + 7168*A^7*B^3 + 64512*A^4*B^5 + 229376*A*B^7)*x - A^12 + 160*A^9*B^2 + 3328*A^6*B^4 +

# p-adic Logarithm

In [25]:
p=5
phi=[1]
R.<T>=Qp(5)[]
for n in range(1,7):
    phi=phi+[R(cyclotomic_polynomial(p^n))]


In [28]:
((phi[2](1+T)/p)*(phi[4](1+T)/p)/p*(phi[6](1+T)/p)%(T^4))

(4 + 3*5 + 5^2 + 3*5^3 + 2*5^4 + 2*5^5 + 2*5^6 + 5^7 + 4*5^8 + 4*5^11 + 4*5^12 + 2*5^13 + 3*5^14 + O(5^18))*T^3 + (4 + 2*5 + 4*5^2 + 3*5^3 + 4*5^4 + 5^5 + 5^6 + 4*5^7 + 3*5^9 + O(5^18))*T^2 + (2 + 2*5^2 + 2*5^4 + O(5^18))*T + 5^-1 + O(5^18)

In [10]:
R(cyclotomic_polynomial(p))

T^20 + T^15 + T^10 + T^5 + 1

In [9]:
def logplus(p,k):
    R.<T>=Qp(p)[]
    partial_product=1
    for n in range(1,k):
        phi = R(cyclotomic_polynomial(p^(2*n)))
        phi_sub=phi(1+T)%(T^2)
        partial_product *= phi_sub/ p
    return(partial_product%(T^2))

In [11]:
logplus(5,4)

(2*5 + 2*5^3 + 2*5^5 + O(5^19))*T + 1 + O(5^19)

In [15]:
p=5
n=4
R.<T>=Qp(p)[]
phi = cyclotomic_polynomial(p^(2*n))
phi

x^312500 + x^234375 + x^156250 + x^78125 + 1